---
title: "Post 4 - Implementing Perceptron"
author: Col McDermott
date: "03-26-2025"
description: "An introductory exploration of the Perceptron algorithm -- FINISH."
format: html
code-fold: true
execute:
  warning: false
  message: false
---

# Abstract

The primary goal of this brief study on the perceptron algorithm is to explore and investigate the processes under the hood of the perceptron algorithm $-$ one of the oldest machine learning algorithms to exist.  The functionality of and logic behind the perceptron algorithm is a backbone to many modern ML methods and models.  It is crucial to develop at least a basic understanding of how perceptron works and why it's design is as such.  This introductory dive into the inner-workings of perceptron involves examining the conditions in which the algorithm is successful, the conditions in which the algorithm must be manually adjusted to prevent non-convergence, various ways the algorithm can be refined to operate on more complex data, and the general limitations and implications associated with this algorithm.

### Generating Data

In [4]:
# Including all additional imports
from sklearn.model_selection import cross_val_score, train_test_split
from matplotlib.colors import LinearSegmentedColormap
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
from sklearn.svm import SVC
import seaborn as sns
import torch as tch
import pandas as pd
import numpy as np
import itertools

# Porting over perceptron implementation
%load_ext autoreload
%autoreload 2
from perceptron import Perceptron, PerceptronOptimizer

# Generating the data - Some code borrowed from Prof. Chodrow
## Linearly separable 2D data
y1 = tch.arange(500) >= int(500 / 2)
X1 = y1[:, None] + tch.normal(0.0, 0.2, size = (500, 2))
X1 = tch.cat((X1, tch.ones((X1.shape[0], 1))), 1)

## Not linearly separable 2D data
y2 = tch.arange(500) >= int(500 / 2)
X2 = y2[:, None] + tch.normal(0.0, 0.3, size = (500, 2))
X2 = tch.cat((X2, tch.ones((X2.shape[0], 1))), 1)

## 6D data
y3 = tch.arange(500) >= int(500 / 2)
X3 = y3[:, None] + tch.normal(0.0, 0.2, size = (500, 6))
X3 = tch.cat((X3, tch.ones((X3.shape[0], 1))), 1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


*Code above generates three random data sets: 1. Linearly separable data with two features.  2. Not linearly separable data with two features.  3. Possibly linearly separable data with 6 features*

To test and investigate the perceptron algorithm, I have created three random datasets.  The first data set has two features and is intentionally linearly separable $-$ the perceptron algorithm should converge to a loss of 0 for this dataset.  The second data set also has two features but is intentionally *not* linearly separable $-$ the perceptron algorithm will not be able to converge to a loss of 0 for this data set and will need to be manually terminated after a certain number of iterations.  The third data set has 6 features and is used to show how the perceptron works with data that can't be easily visualized $-$ this dataset is possibly linearly separable.

### Implementing the Perceptron Algorithm

For this introductory study, I have implemented a rudimentary version of the perceptron algorithm.  This implementation involves three class definitions: `LinearModel`,   `Perceptron`, and `PerceptronOptimizer`.

**`LinearModel`**:
-   `self.w`: An instance variable to store the weights vector of a linear model
-   `score(X)`: Method to compute the score $s_i$ for each data point in the feature matrix **$X$**
-   `predict(X)`: Method to compute the classification prediction $\hat{y}_i$ $\in\{0, 1\}$ for each data point

**`Perceptron` (inherits from `LinearModel`)**:
-   `loss(X, y)`: Method to compute the misclassification rate in the data $-$ A point i is classified correctly if $s_i\bar{y}_i > 0$, where $\bar{y}_i \in \{-1, 1\}$ is the modified classification label.
-   `grad(x, y)`: Method to compute the perceptron update for a sampled data point
    -   This method takes as arguments `x` $-$ the row of the feature matrix **$X$** corresponding to the sampled data point $-$ and `y` $-$ the classification target vector
    -   This method first computes the score $s_i$ of the sampled data point with $<$**$w$**$, x_i>$
    -   This method then computes the vector $-I[s_i(2y_i - 1) < 0]y_ix_i$ which represents the perceptron update (moving the score $s_i$ closer to the target $y_i$) with respect to the sampled data point.
    -   Ultimately, this method computes an update to a sampled data point that later adjusts the weight vector of the perceptron algorithm to better fit the sampled data point

### Evaluating the Perceptron Implementation

In [5]:
# Code provided by Prof. Chodrow

# Instantiate a model and an optimizer
p = Perceptron()
opt = PerceptronOptimizer(p)

# Initialize the loss
loss = 1.0

# Keeping track of loss values
loss_vec = []

# Iteration counter
j = 0
n = X1.size()[0]
while (loss > 0) and (j < 1000):
    
    # not part of the update: just for tracking our progress    
    loss = p.loss(X1, y1) 
    loss_vec.append(loss)
    
    # pick a random data point
    i = tch.randint(n, size = (1,))
    x_i = X1[[i],:]
    y_i = y1[i]
    
    # perform a perceptron update using the random data point
    opt.step(x_i, y_i)
    j += 1

# Observe the algorithm's performance
print("Changing loss values:\n")
for i in range(len(loss_vec)):
    print(f"Iteration: {i} | Loss: {loss_vec[i]}")

RuntimeError: output with shape [3] doesn't match the broadcast shape [1, 3]

*Code above checks the implementation of the perceptron algorithm*

### Experimenting With Perceptron

##### Linearly Separable Data

*Code description*

Analysis of perceptron on linearly separable data.

##### Not Linearly Separable Data

*Code description*

Analysis of perceptron on not linearly separable data.

##### Higher-Dimensional Data

*Code description*

Analysis of perceptron on higher dimensional, possibly linearly separable data.

### Minibatch Perceptron

*Code description*

Exploration of the minibatch perceptron algorithm.

##### Minibatch Perceptron Experiments

*Code Description*

Evaluation of the minibatch perceptron algorithm.

### Concluding Thoughts

Analysis of the findings from this study.

*During the implementation process of this replication study, I collaborated with _____.*